In [ ]:
import polars as pl
import numpy as np
import networkx as nx
import random
from math import cos, sin, pi ,sqrt
import rtsvg
rt  = rtsvg.RACETrack()
# Circles
#circles     = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6),
#               (100,20,10),(80,15,5)] # additions from earlier versions of this structure
circles     = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6)]
svg_hdr     = f'<svg x="0" y="0" width="512" height="512" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white"/>'
svg_circles = []
for i in range(len(circles)): svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" stroke="#000000" fill="none"/>')
svg_ftr     = '</svg>'

In [2]:
# assumes that the circles are nowhere near linearly arranged
def poorThreeCirclesMeet(c0, c1, c2):
    # 0-1 bisector
    uv01           = rt.unitVector((c0,c1))
    xy0_01         = (c0[0] + uv01[0]*c0[2], c0[1] + uv01[1]*c0[2])
    xy1_01         = (c1[0] - uv01[0]*c1[2], c1[1] - uv01[1]*c1[2])
    bisector_01    = ((xy0_01[0] + xy1_01[0])/2, (xy0_01[1] + xy1_01[1])/2)
    bisector_01_uv = (uv01[1], -uv01[0])
    # 1-2 bisector
    uv12           = rt.unitVector((c1,c2))
    xy1_12         = (c1[0] + uv12[0]*c1[2], c1[1] + uv12[1]*c1[2])
    xy2_12         = (c2[0] - uv12[0]*c2[2], c2[1] - uv12[1]*c2[2])
    bisector_12    = ((xy1_12[0] + xy2_12[0])/2, (xy1_12[1] + xy2_12[1])/2)
    bisector_12_uv = (uv12[1], -uv12[0])
    # 2-0 bisector
    uv20           = rt.unitVector((c2,c0))
    xy2_20         = (c2[0] + uv20[0]*c2[2], c2[1] + uv20[1]*c2[2])
    xy0_20         = (c0[0] - uv20[0]*c0[2], c0[1] - uv20[1]*c0[2])
    bisector_20    = ((xy2_20[0] + xy0_20[0])/2, (xy2_20[1] + xy0_20[1])/2)
    bisector_20_uv = (uv20[1], -uv20[0])
    # intersection of bisectors
    inter_01_12 = rt.intersectionPoint((bisector_01, (bisector_01[0] + bisector_01_uv[0], bisector_01[1] + bisector_01_uv[1])),
                                       (bisector_12, (bisector_12[0] + bisector_12_uv[0], bisector_12[1] + bisector_12_uv[1])))
    inter_12_20 = rt.intersectionPoint((bisector_12, (bisector_12[0] + bisector_12_uv[0], bisector_12[1] + bisector_12_uv[1])),
                                       (bisector_20, (bisector_20[0] + bisector_20_uv[0], bisector_20[1] + bisector_20_uv[1])))
    inter_20_01 = rt.intersectionPoint((bisector_20, (bisector_20[0] + bisector_20_uv[0], bisector_20[1] + bisector_20_uv[1])),
                                       (bisector_01, (bisector_01[0] + bisector_01_uv[0], bisector_01[1] + bisector_01_uv[1])))
    # average of the three intersections
    return (inter_01_12[0] + inter_12_20[0] + inter_20_01[0])/3.0, (inter_01_12[1] + inter_12_20[1] + inter_20_01[1])/3.0

def pointWithinTriange(triangle, pt):
    x_min, y_min, x_max, y_max = triangle[0][0], triangle[0][1], triangle[0][0], triangle[0][1]
    for i in range(1,3): x_min, y_min, x_max, y_max = min(x_min, triangle[i][0]), min(y_min, triangle[i][1]), max(x_max, triangle[i][0]), max(y_max, triangle[i][1])
    if pt[0] < x_min or pt[0] > x_max or pt[1] < y_min or pt[1] > y_max: return False
    # Based on description found at https://nerdparadise.com/math/pointinatriangle
    def crossProduct(p0,p1,p2): return (p1[0] - p0[0]) * (p2[1] - p0[1]) - (p1[1] - p0[1]) * (p2[0] - p0[0])
    cp0 = crossProduct(pt, triangle[0], triangle[1])
    cp1 = crossProduct(pt, triangle[1], triangle[2])
    cp2 = crossProduct(pt, triangle[2], triangle[0])
    if cp0 < 0 and cp1 < 0 and cp2 < 0:         return True
    if cp0 > 0 and cp1 > 0 and cp2 > 0:         return True
    if (cp0 == 0 and cp1 == 0) or \
       (cp1 == 0 and cp2 == 0) or \
       (cp2 == 0 and cp0 == 0):                 return True
    if (cp0 == 0 and cp1 > 0 and cp2 > 0) or \
       (cp1 == 0 and cp2 > 0 and cp0 > 0) or \
       (cp2 == 0 and cp0 > 0 and cp1 > 0):      return True
    return False

csample = [(25,25,20),(80,80,40),(20,100,10)]
svg_csample = []
for i in range(len(csample)): svg_csample.append(f'<circle cx="{csample[i][0]}" cy="{csample[i][1]}" r="{csample[i][2]}" stroke="#000000" fill="none"/>')
svg_grid    = []
for x in range(0,128):
    for y in range(0,128):
        d0 = sqrt((x-csample[0][0])**2 + (y-csample[0][1])**2) - csample[0][2]
        d1 = sqrt((x-csample[1][0])**2 + (y-csample[1][1])**2) - csample[1][2]
        d2 = sqrt((x-csample[2][0])**2 + (y-csample[2][1])**2) - csample[2][2]
        d  = abs(d2 - d1) + abs(d0 - d1) + abs(d0 - d2)
        if d < 20.0:
            _color_ = rt.co_mgr.spectrum(d, 0.0, 20.0) 
            svg_grid.append(f'<rect x="{x}" y="{y}" width="1" height="1" fill="{_color_}"/>')
for i in range(len(csample)):
    for j in range(len(csample)):
        if i == j: continue
        c0    = csample[i]
        c1    = csample[j]
        uv    = rt.unitVector((c0,c1))
        c0_xy = (c0[0] + uv[0]*c0[2], c0[1] + uv[1]*c0[2])
        c1_xy = (c1[0] - uv[0]*c1[2], c1[1] - uv[1]*c1[2])
        svg_grid.append(f'<line x1="{c0_xy[0]}" y1="{c0_xy[1]}" x2="{c1_xy[0]}" y2="{c1_xy[1]}" stroke="#ff0000" stroke-width="0.05"/>')
        bisector_xy = ((c0_xy[0] + c1_xy[0])/2, (c0_xy[1] + c1_xy[1])/2)
        #svg_grid.append(f'<circle cx="{bisector_xy[0]}" cy="{bisector_xy[1]}" r="0.1" stroke="#000000" fill="none"/>')
        uv_perp = (uv[1], -uv[0])
        svg_grid.append(f'<line x1="{bisector_xy[0]}" y1="{bisector_xy[1]}" x2="{bisector_xy[0]+uv_perp[0]*100}" y2="{bisector_xy[1]+uv_perp[1]*100}" stroke="#0000ff" stroke-width="0.2"/>')
_xy_ = poorThreeCirclesMeet(csample[0], csample[1], csample[2])
svg_grid.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="2.0" stroke="#000000" fill="none"/>')
#rt.tile([svg_hdr + ''.join(svg_grid) + ''.join(svg_csample) + svg_ftr])

In [ ]:
# Polygons
polys = rt.isedgarVoronoi(circles, ((0.0,0.0),(0.0,128.0),(128.0,128.0),(128.0,0.0)), use_circle_radius=True)
svg_polys, xy_seen = [], set()
for i in range(len(polys)):
    _poly_ = polys[i]
    d      = f'M {_poly_[0][0]} {_poly_[0][1]} '
    for j in range(1,len(_poly_)): d += f'L {_poly_[j][0]} {_poly_[j][1]} '
    d     += f'Z'
    svg_polys.append(f'<path d="{d}" stroke="#000000" stroke-width="0.1" fill="none"/>')
    for j in range(len(_poly_)):
        _xy_ = _poly_[j]
        if _xy_ not in xy_seen: svg_polys.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{0.1+1.1*random.random()}" stroke="#000000" stroke-width="0.1" fill="none" />')
        xy_seen.add(_xy_)

def containsAnotherCircle(c0, c1, c2):
    for _circle_ in circles:
        if _circle_ == c0 or _circle_ == c1 or _circle_ == c2: continue
        if pointWithinTriange((c0,c1,c2), _circle_): return True
    return False

# For all circles, draw the points between the three circles
svg_inters = []
for i in range(len(circles)):
    c0 = circles[i]
    for ii in range(i+1,len(circles)):
        c1 = circles[ii]
        for iii in range(ii+1,len(circles)):
            c2 = circles[iii]
            if i != ii and ii != iii and i != iii: # not the same circle
                if (c1[1] - c0[1])*(c2[0] - c1[0]) != (c2[1] - c1[1]) * (c1[0] - c0[0]): # not colinear
                    if containsAnotherCircle(c0, c1, c2): continue
                    _xy_ = poorThreeCirclesMeet(c0, c1, c2)
                    svg_inters.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="0.5" stroke="none" fill="#ff0000"/>')
               
rt.tile([svg_hdr + ''.join(svg_polys) + ''.join(svg_circles) + ''.join(svg_inters) + svg_ftr])